In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.simplefilter("ignore")

In [2]:
df=pd.read_csv("CleanedLoanData.csv")

In [3]:
x=df.drop("loan_status",axis=1)
y=df["loan_status"]

In [4]:
contnious=["person_age","person_income","loan_amnt","loan_int_rate","loan_percent_income",
          "credit_score"]

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=77)

In [6]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train[contnious]=sc.fit_transform(x_train[contnious])
x_test[contnious]=sc.transform(x_test[contnious])

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

#Modeling
model=DecisionTreeClassifier(random_state=True)
model.fit(x_train,y_train)

#Prediction
yPredTrain=model.predict(x_train)

#Accuracy
TrainAccuracy=accuracy_score(y_train,yPredTrain)

# CV
TrainCV=cross_val_score(model,x_train,y_train,cv=5,scoring="accuracy").mean()

# print O/P
print("TrainAccuracy: ",TrainAccuracy," ","TrainCV: ",TrainCV)

TrainAccuracy:  1.0   TrainCV:  0.9009334012439998


In [8]:
model.get_depth()

31

In [20]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import GridSearchCV
# estimator=DecisionTreeClassifier
# param_grid={'criterion':["gini","entropy"],"max_depth":len(range(1,32))}
# grid=GridSearchCV(estimator,param_grid,cv=5,scoring="accuracy")
# grid.fit(x_train,y_train)
# grid.best_params_



from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
esstimator=DecisionTreeClassifier(random_state=True)

param_grid={"criterion":["gini","entropy"],"max_depth": list(range(1,32))}
grid=GridSearchCV(esstimator,param_grid,cv=5,scoring="accuracy")
grid.fit(x_train,y_train)
grid.best_params_

{'criterion': 'entropy', 'max_depth': 11}

In [34]:
grid.best_estimator_.feature_importances_

array([0.00444573, 0.00103259, 0.08527327, 0.00199459, 0.00643205,
       0.14168888, 0.13929878, 0.00240951, 0.01919713, 0.00090557,
       0.        , 0.01229939, 0.05657287, 0.00232759, 0.00797231,
       0.00668632, 0.00173715, 0.01277271, 0.49695357])

In [36]:
impDF=pd.DataFrame(grid.best_estimator_.feature_importances_,
                  columns=["important_feature"],
                  index=x_train.columns.tolist())
impDF

,important_feature
person_age,0.004446
person_education,0.001033
person_income,0.085273
person_emp_exp,0.001995
loan_amnt,0.006432
loan_int_rate,0.141689
loan_percent_income,0.139299
cb_person_cred_hist_length,0.002410
credit_score,0.019197
person_gender_male,0.000906


In [42]:
impCol=impDF[impDF["important_feature"]>0].index.tolist()

In [52]:
x_train=x_train[impCol]
x_test=x_test[impCol]


In [56]:
#modeling
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(criterion='entropy',max_depth=11,random_state=True)
model.fit(x_train,y_train)

#prediction
Y_predTrain=model.predict(x_train)
Y_predTest=model.predict(x_test)

#Evalution
from sklearn.metrics import accuracy_score 
accuracyTrain=accuracy_score(y_train,Y_predTrain)
accuracyTest=accuracy_score(y_test,Y_predTest)

#CV
from sklearn.model_selection import cross_val_score
TrainCV=cross_val_score(model,x_train,y_train,cv=5,scoring="accuracy").mean()

print("accuracyTrain: ",accuracyTrain," ","accuracyTest: ",accuracyTest," ","TrainCV: " ,TrainCV)


accuracyTrain:  0.9347705300588954   accuracyTest:  0.9167685298366485   TrainCV:  0.921491252643114
